In [17]:
import json, os, pprint, time
from urllib import parse
import requests
from bs4 import BeautifulSoup
import re

'''放置 metadata 的資訊'''
listData = []

'''古騰堡計劃的網址'''
url = 'https://www.gutenberg.org/browse/languages/zh'
url1 = 'https://www.gutenberg.org/files/'

'''設定標頭'''
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
}

# 取得文件的主要連結 , 並直接轉成聯結檔
def getMainLinks():
    response = requests.get(url, headers = headers)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, "lxml")
    a_elms = soup.select('div.pgdbbylanguage li a')
    for a in a_elms:
        link = (a.get('href')).replace("/ebooks/", "")
#         specialChars = "\n\=/:*<>|\r"
#         for specialChar in specialChars :
        listData.append({            
            "title": a.text.replace("/", ""),
#             "link": a.get('href')
            "link": url1 + link +"/"+ link +"-0.txt"
        })
#         listData = listData.replace(" ", "")
#         listData = listData.replace("\r", "")
#         listData = listData.replace("\n", "")
#         listData = listData.replace("　", "")
        print(a_elms)


# # 建立 metadata 的 json 檔
def saveJson():
    fp = open("gutenberg4.json", "w", encoding="utf-8")
    fp.write( json.dumps(listData, ensure_ascii=False) )
    fp.close()

# # 將古籍經典所有內容，各自寫到 txt 與 json 中
def writeTxt():
    # 稍候建立 train.json 前的程式變數
    listContent = []

    # 開啟 古籍經典 metadata 的 json 檔
    fp = open("gutenberg4.json", "r", encoding="utf-8")
    strJson = fp.read()
    fp.close()

    # 沒有放置 txt 檔的資料夾，就建立起來
    folderPath = 'gutenberg4_txt'
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)

    # 走訪所有古籍經典文字內容
    listResult = json.loads(strJson)
    for i in range(len(listResult)):
        response = requests.get(listResult[i]["link"], headers = headers)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, "lxml")
        strContent = soup.p
        # 資料清理
        regex01 = re.compile(r'[\u4e00-\u9fff，。：「」；、？！『』]+')
        strContent = ''.join(regex01.findall(strContent.text))
       

        # 決定 txt 的檔案名稱
        fileName = f"{listResult[i]['title']}.txt"
            
        # 將古籍經典內容存到 txt 中
        fp = open(f"{folderPath}/{fileName}", "w", encoding="utf-8")
        fp.write(strContent)
        fp.close()

if __name__ == "__main__":
#     time1 = time.time()
#     getMainLinks()
#     saveJson()
    writeTxt()
#     print(f"執行總花費時間: {time.time() - time1}")


FileNotFoundError: [Errno 2] No such file or directory: 'gutenberg4_txt/True Heart/Mind.txt'